#### Importing Libraries

In [1]:
import pandas as pd
import numpy as np
from math import log, sqrt
from sklearn.metrics import mean_squared_error

#### Import Data

In [2]:
dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float, 'grade':int, 'yr_renovated':int, 'price':float, 'bedrooms':float, 'zipcode':str, 'long':float, 'sqft_lot15':float, 'sqft_living':float, 'floors':float, 'condition':int, 'lat':float, 'date':str, 'sqft_basement':int, 'yr_built':int, 'id':str, 'sqft_lot':int, 'view':int}

In [62]:
sales = pd.read_csv('kc_house_data.csv ', dtype=dtype_dict)
train_data = pd.read_csv('kc_house_train_data.csv', dtype=dtype_dict)
test_data = pd.read_csv('kc_house_test_data.csv' , dtype=dtype_dict)

In [63]:
print(sales.head())
print(train_data.head())
print(test_data.head())

           id             date     price  bedrooms  bathrooms  sqft_living  \
0  7129300520  20141013T000000  221900.0       3.0       1.00       1180.0   
1  6414100192  20141209T000000  538000.0       3.0       2.25       2570.0   
2  5631500400  20150225T000000  180000.0       2.0       1.00        770.0   
3  2487200875  20141209T000000  604000.0       4.0       3.00       1960.0   
4  1954400510  20150218T000000  510000.0       3.0       2.00       1680.0   

   sqft_lot  floors  waterfront  view     ...      grade  sqft_above  \
0      5650     1.0           0     0     ...          7        1180   
1      7242     2.0           0     0     ...          7        2170   
2     10000     1.0           0     0     ...          6         770   
3      5000     1.0           0     0     ...          7        1050   
4      8080     1.0           0     0     ...          8        1680   

   sqft_basement  yr_built  yr_renovated  zipcode      lat     long  \
0              0      1955 

#### Adding Constant column to Feature Matrices

In [5]:
def get_numpy_data(data, features, output):
    data['constant'] = 1 # add a constant column - for intercept term
    
    # prepend variable 'constant' to the features list
    features = ['constant'] + features
    
    # this will convert the features_sframe into a numpy matrix with GraphLab Create >= 1.7!!
    features_matrix = np.array(data.loc[:,features])
 

    # this will convert the SArray into a numpy array:
    output_array = np.array(data.loc[:,output]) 
    return(features_matrix, output_array)


In [6]:
def predict_output(feature_matrix, weights):
    predictions = np.dot(feature_matrix, weights)
    return(predictions)

#### Normalization

In [7]:
def normalize_features(features):
    norms = np.linalg.norm(features, axis=0) ### sqrt(f1^2 + f2^2 + ......... + fd^2)
    normalized_features = features / norms
    return (normalized_features, norms)

#### Simple Model

In [8]:
simple_features = ['sqft_living', 'bedrooms']
output = ['price']
simple_features_matrix, simple_output = get_numpy_data(sales, simple_features, output)

In [9]:
print(simple_features_matrix)
print(simple_output)

[[  1.00000000e+00   1.18000000e+03   3.00000000e+00]
 [  1.00000000e+00   2.57000000e+03   3.00000000e+00]
 [  1.00000000e+00   7.70000000e+02   2.00000000e+00]
 ..., 
 [  1.00000000e+00   1.02000000e+03   2.00000000e+00]
 [  1.00000000e+00   1.60000000e+03   3.00000000e+00]
 [  1.00000000e+00   1.02000000e+03   2.00000000e+00]]
[[ 221900.]
 [ 538000.]
 [ 180000.]
 ..., 
 [ 402101.]
 [ 400000.]
 [ 325000.]]


#### Normalizing Input

In [10]:
simple_features_matrix_norm, norms = normalize_features(simple_features_matrix)

In [11]:
print(simple_features_matrix_norm)
print(norms)

[[ 0.00680209  0.00353021  0.00583571]
 [ 0.00680209  0.00768869  0.00583571]
 [ 0.00680209  0.00230361  0.00389048]
 ..., 
 [ 0.00680209  0.00305154  0.00389048]
 [ 0.00680209  0.00478673  0.00583571]
 [ 0.00680209  0.00305154  0.00389048]]
[  1.47013605e+02   3.34257264e+05   5.14075870e+02]


#### Cordinate Descent

In [12]:
def lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty):
    
    weights = np.array(weights)
    
    prediction = predict_output(feature_matrix, weights)
    
    # compute ro[i] = SUM[ [feature_i]*(output - prediction + weight[i]*[feature_i]) ]
    ro_i = np.sum((feature_matrix[:,i]) * (output.reshape(len(output),) - prediction + np.dot(feature_matrix[:,i], weights[i])))
    
    if i == 0: # intercept -- do not regularize
        new_weight_i = ro_i
    elif ro_i < -l1_penalty/2.:
        new_weight_i = ro_i + l1_penalty/2
    elif ro_i > l1_penalty/2.:
        new_weight_i = ro_i - l1_penalty/2
    else:
        new_weight_i = 0.
    
    return new_weight_i

#### Qn 1

In [13]:
initial_weights = [1., 4., 1.]
predict = predict_output(simple_features_matrix_norm, initial_weights)
ro_i_feature1 = np.sum((simple_features_matrix_norm[:,1]) * (simple_output.reshape(len(simple_output),) - predict + np.dot(simple_features_matrix_norm[:,1], initial_weights[1])))

In [14]:
print(2*ro_i_feature1/1e8)

1.75878941647


#### Qn 2

In [15]:
ro_i_feature2 = np.sum((simple_features_matrix_norm[:,2]) * (simple_output.reshape(len(simple_output),) - predict + np.dot(simple_features_matrix_norm[:,2], initial_weights[2])))

In [16]:
print(2*ro_i_feature2/1e8)

1.61933397332


#### Test

In [17]:
# should print 0.425558846691
import math
print(lasso_coordinate_descent_step(1, np.array([[3./math.sqrt(13),1./math.sqrt(10)],
                   [2./math.sqrt(13),3./math.sqrt(10)]]), np.array([1., 1.]), np.array([1., 4.]), 0.1))

0.425558846691


#### Cyclical Coordinate Descent

In [213]:
def lasso_cyclical_coordinate_descent(feature_matrix, output, initial_weights, l1_penalty, tolerance):
    
    initial_weights = np.array(initial_weights)
    
    while True:
        max_diff = 0
        for i in range(feature_matrix.shape[1]):
            weights = (lasso_coordinate_descent_step(i, feature_matrix, output, initial_weights, l1_penalty))
            if np.absolute(weights - initial_weights[i]) > max_diff:
                max_diff = np.absolute(weights - initial_weights[i])
            initial_weights[i] = weights
        
        if max_diff < (tolerance * l1_penalty):
            break
    
    return initial_weights

#### Qn 3

In [210]:
initial_weights = [0.]*3
l1_penalty = 1e7
tolerance = 1.0

In [211]:
final_weights = lasso_cyclical_coordinate_descent(simple_features_matrix_norm, simple_output, initial_weights, l1_penalty, tolerance)

In [214]:
final_weights

array([ 70262136.26121683,  18947595.76476732,         0.        ])

In [215]:
residue = predict_output(simple_features_matrix_norm, final_weights) - simple_output.reshape(len(simple_output),)
residue

array([ 322918.57015868,   85611.64645016,  341577.44693601, ...,
        133647.8635352 ,  168626.55004532,  210748.8635352 ])

In [216]:
rss = np.sum(residue ** 2)/1e15
rss

2.4586552174937886

#### Qn 4

In [174]:
final_weights

array([ 70262136.26121683,  18947595.76476732,         0.        ])

#### Qn 5

In [64]:
new_features = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated']
output = ['price']
train_features_matrix, output_train = get_numpy_data(train_data, new_features, output)
test_features_matrix, output_test = get_numpy_data(test_data, new_features, output)

In [70]:
print(train_features_matrix)
print(output_train)

[[  1.00000000e+00   3.00000000e+00   1.00000000e+00 ...,   0.00000000e+00
    1.95500000e+03   0.00000000e+00]
 [  1.00000000e+00   3.00000000e+00   2.25000000e+00 ...,   4.00000000e+02
    1.95100000e+03   1.99100000e+03]
 [  1.00000000e+00   2.00000000e+00   1.00000000e+00 ...,   0.00000000e+00
    1.93300000e+03   0.00000000e+00]
 ..., 
 [  1.00000000e+00   3.00000000e+00   2.50000000e+00 ...,   0.00000000e+00
    2.00900000e+03   0.00000000e+00]
 [  1.00000000e+00   3.00000000e+00   2.50000000e+00 ...,   0.00000000e+00
    2.00400000e+03   0.00000000e+00]
 [  1.00000000e+00   2.00000000e+00   7.50000000e-01 ...,   0.00000000e+00
    2.00800000e+03   0.00000000e+00]]
[[ 221900.]
 [ 538000.]
 [ 180000.]
 ..., 
 [ 360000.]
 [ 400000.]
 [ 325000.]]


In [69]:
print(test_features_matrix)
print(output_test)

[[  1.00000000e+00   3.00000000e+00   1.00000000e+00 ...,   0.00000000e+00
    1.92700000e+03   0.00000000e+00]
 [  1.00000000e+00   4.00000000e+00   3.00000000e+00 ...,   9.70000000e+02
    1.97900000e+03   0.00000000e+00]
 [  1.00000000e+00   3.00000000e+00   2.00000000e+00 ...,   0.00000000e+00
    1.94100000e+03   0.00000000e+00]
 ..., 
 [  1.00000000e+00   4.00000000e+00   2.50000000e+00 ...,   0.00000000e+00
    2.01400000e+03   0.00000000e+00]
 [  1.00000000e+00   4.00000000e+00   2.50000000e+00 ...,   0.00000000e+00
    2.01400000e+03   0.00000000e+00]
 [  1.00000000e+00   2.00000000e+00   7.50000000e-01 ...,   0.00000000e+00
    2.00900000e+03   0.00000000e+00]]
[[ 310000.]
 [ 650000.]
 [ 233000.]
 ..., 
 [ 610685.]
 [ 400000.]
 [ 402101.]]


#### Normalizing Train Data

In [217]:
train_features_matrix_norm, train_norms = normalize_features(train_features_matrix)

In [218]:
print(train_features_matrix_norm)
print(train_norms)

[[ 0.00758447  0.00652117  0.0033687  ...,  0.          0.00752148  0.        ]
 [ 0.00758447  0.00652117  0.00757957 ...,  0.0057043   0.0075061
   0.03707954]
 [ 0.00758447  0.00434745  0.0033687  ...,  0.          0.00743684  0.        ]
 ..., 
 [ 0.00758447  0.00652117  0.00842175 ...,  0.          0.00772924  0.        ]
 [ 0.00758447  0.00652117  0.00842175 ...,  0.          0.00771     0.        ]
 [ 0.00758447  0.00434745  0.00252652 ...,  0.          0.00772539  0.        ]]
[  1.31848398e+02   4.60040216e+02   2.96850552e+02   2.99962419e+05
   5.81709718e+06   2.09458827e+02   1.15325626e+01   1.05933942e+02
   4.57793622e+02   1.02101959e+03   2.59726472e+05   7.01224951e+04
   2.59922094e+05   5.36953839e+04]


In [219]:
initial_weights = [0.]*14
l1_penalty = 1e7
tolerance = 1.0

In [220]:
weights_1e7 = lasso_cyclical_coordinate_descent(train_features_matrix_norm, output_train, initial_weights, l1_penalty, tolerance)
weights_1e7

array([ 66090269.34433953,         0.        ,   5640929.98980149,
         9576074.11132805,         0.        ,         0.        ,
         4372018.71148063,   8681305.93741526,  -5916544.49084155,
               0.        ,         0.        ,   2344289.5470792 ,
        -2427081.43932464,         0.        ])

In [ ]:
bathrooms, sqft_living, waterfront, view, condition, sqft_basement, yr_built
bedrooms, bathrooms, sqft_living, sqft_lot, floors, waterfront, view, condition, grade, sqft_above, sqft_basement, yr_built, yr_renovated

In [201]:
2344289.5470792/0.5e7

0.46885790941584

#### Qn 6

In [180]:
l1_penalty = 1e8

In [181]:
weights_1e8 = lasso_cyclical_coordinate_descent(train_features_matrix_norm, output_train, initial_weights, l1_penalty, tolerance)
weights_1e8

array([ 71114625.71488702,         0.        ,         0.        ,
               0.        ,         0.        ,         0.        ,
               0.        ,         0.        ,         0.        ,
               0.        ,         0.        ,         0.        ,
               0.        ,         0.        ])

#### Qn 7

In [182]:
l1_penalty = 1e4

In [183]:
weights_1e4 = lasso_cyclical_coordinate_descent(train_features_matrix_norm, output_train, initial_weights, l1_penalty, tolerance)
weights_1e4

array([  7.00051156e+08,  -1.94071020e+07,   1.11156277e+07,
         5.24876410e+07,  -1.37873317e+06,   3.85963584e+06,
         6.84667148e+06,   4.65892230e+06,   1.03636103e+07,
         1.26466051e+08,   0.00000000e+00,   5.35299481e+05,
        -8.09539500e+08,   9.70282024e+05])

#### Qn 8

In [191]:
### 1e7 penalty

weights_1e7_norm = weights_1e7/train_norms

In [193]:
residue_1e7 = predict_output(test_features_matrix, weights_1e7_norm) - output_test.reshape(len(output_test),)
residue_1e7

array([ 186223.8489603 ,  223471.32658052,  278110.45888695, ...,
        -39048.17341711,  164932.73487634,   88441.60945701])

In [194]:
rss_1e7 = np.sum(residue_1e7 ** 2)/1e15
rss_1e7

0.38988733983922685

In [195]:
### 1e8 penalty

weights_1e8_norm = weights_1e8/train_norms

In [196]:
residue_1e8 = predict_output(test_features_matrix, weights_1e8_norm) - output_test.reshape(len(output_test),)
residue_1e8

array([ 229366.62793373, -110633.37206627,  306366.62793373, ...,
        -71318.37206627,  139366.62793373,  137265.62793373])

In [197]:
rss_1e8 = np.sum(residue_1e8 ** 2)/1e15
rss_1e8

0.53716615149732272

In [198]:
### 1e4 penalty

weights_1e4_norm = weights_1e4/train_norms

In [199]:
residue_1e4 = predict_output(test_features_matrix, weights_1e4_norm) - output_test.reshape(len(output_test),)
residue_1e4

array([ 139408.48588709,  313302.71605342,  161623.85203612, ...,
         10058.52798233,   60184.83733669, -256031.04904917])

In [200]:
rss_1e4 = np.sum(residue_1e4 ** 2)/1e15
rss_1e4

0.19444810029655984